# N01-Transcript_to_facts
*Purpose*: To convert raw extracted shark tank episode transcripts (from derrick) into facts dictionary which contains 5 portion. The 5 portion can then be used as our dataset for downstream models

    1. Facts about the company or product that are hard quantifiable facts
    2. Product description
    3. Summary of investor pitch. This should contain information about how the pitch was delivered, the sentiment, the story, and other aspects essential to the pitch.
    4. Entrepreneur initial offer
    5. Final agreed offer by the shark investor

In [ ]:
# run this cell if using google colab (kaizhuo G drive)
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/MyDrive/MITB/CS6xx\ LLM/LLM\ Project

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/MyDrive/MITB/CS6xx LLM/LLM Project


In [3]:
pip install transformers datasets evaluate torch accelerate datasets together openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [4]:
# Import required libraries
import torch
import time
import requests
import re
import json
import numpy as np
import pandas as pd
from transformers import pipeline
from datasets import load_dataset
from torch.nn import functional as F
from transformers import AutoTokenizer
from together import Together
from openai import OpenAI
from pathlib import Path

In [5]:
client = OpenAI(api_key="sk-5fb8c6b1d15f4ab1bb4e218980869d8c", base_url="https://api.deepseek.com")

def transcript_to_facts(transcript):
    response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
            {"role": "system",
            "content":
            """
    In this transcript of a shark tank episode, can you extract out this 4 category of information?

    1. Facts about the company or product that are hard quantifiable facts
    2. Product description
    3. Summary of investor pitch. This should contain information about how the pitch was delivered, the sentiment, the story, and other aspects essential to the pitch.
    4. Entrepreneur initial offer
    5. Final agreed offer by the shark investor

    Convert it into a json format.

    The transcript is below:
    """},
            {"role": "user", "content": transcript},
        ],
        stream=False
    )
    response = response.choices[0].message.content
    response = json.loads(response.strip("```json\n").strip("```"))
    return response

In [10]:
# read all transcripts available
folder_path = Path("./Prelim_Data_Split_Clean")
transcript_store = {}
for file in folder_path.glob("*.txt"):
    with file.open("r", encoding="utf-8") as f:
        transcript_store[file.name] = f.read()

# read all facts that has already been processed (so as to avoid processing them again)
facts_store = {}
folder_path = Path("./facts")
for file in folder_path.glob("*.txt"):
    with file.open("r", encoding="utf-8") as f:
        facts_store[file.name] = f.read()

# loop through each transcript, check that it is not in the facts folder, then process them
facts_file_path = './facts'
for transcript_name, transcript in transcript_store.items():
    print(f'Processing: {transcript_name}')
    if f'facts_{transcript_name}' in facts_store:
        print(f'{transcript_name} has already been processed, skipping ... ...')
        continue
    else:
        timenow = time.time()
        facts = transcript_to_facts(transcript)
        print(f'Saving facts: {transcript_name}. Latency: {time.time()-timenow}')
        with open(f'{facts_file_path}/facts_{transcript_name}', "w", encoding="utf-8") as file:
            json.dump(facts, file, indent=4)  # `indent=4` makes it readable


Processing: shark_tank_transcript_0_TouchUp Cup.txt
shark_tank_transcript_0_TouchUp Cup.txt has already been processed, skipping ... ...
Processing: shark_tank_transcript_0_Roadie.txt
shark_tank_transcript_0_Roadie.txt has already been processed, skipping ... ...
Processing: shark_tank_transcript_0_GarmaGuard.txt
shark_tank_transcript_0_GarmaGuard.txt has already been processed, skipping ... ...
Processing: shark_tank_transcript_1_BootayBag.txt
shark_tank_transcript_1_BootayBag.txt has already been processed, skipping ... ...
Processing: shark_tank_transcript_1_GoOats.txt
shark_tank_transcript_1_GoOats.txt has already been processed, skipping ... ...
Processing: shark_tank_transcript_3_Electra.txt
shark_tank_transcript_3_Electra.txt has already been processed, skipping ... ...
Processing: shark_tank_transcript_3_All33.txt
shark_tank_transcript_3_All33.txt has already been processed, skipping ... ...
Processing: shark_tank_transcript_3_His & Her Bar.txt
shark_tank_transcript_3_His & Her

In [11]:
len(transcript_store)

119